# Importing libraries

In [1]:
import os
import sys
from dotenv import load_dotenv
from typing import Tuple
import torch
from torch.utils.data import Dataset, DataLoader
import wandb
from src.utils import set_seed, load_text, split_text, speedometer
from src.config import ModelConfig, TrainConfig, GenerationConfig
from src.tokenizer import CharTokenizer
from models.GPT import GPT
from src.train import Trainer

/home/pathfinder/miniconda3/envs/torch-env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
PROJECT_ROOT = os.path.abspath(os.getcwd())
sys.path.append(PROJECT_ROOT)
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

PROJECT_ROOT: /home/pathfinder/projects/PathFinder


# Configuration

In [3]:
model_config = ModelConfig(
    vocab_size=-1,
    max_seq_len=128,
    d_embed=128,
    n_layers=4,
    n_heads=4,
    d_head=32,
    rank=16,
    d_ff=-1,
    d_ff_multiple_of=64,
    beta_min=1/2,
    beta_max=8
)

train_config = TrainConfig(
    debug=False,
    wandb_project="nanoGPT",
    model_name="PathFinder",
    per_device_train_batch_size=512,
    per_device_eval_batch_size=1024,
    gradient_accumulation_steps=512 // 512,
    num_train_epochs=1,
    learning_rate=5e-3,
    eval_steps=100,
    mixed_precision=True,
    matmul_precision="high",
)

generation_config = GenerationConfig(
    use_cache=True,
    max_new_tokens=1000,
    temperature=1.0,
    top_k=50
)

In [4]:
load_dotenv()
wandb.login(key=os.environ.get("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pathfinder/.netrc
wandb: Currently logged in as: pathfinderkr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Utils

## Reproducibility

In [5]:
set_seed(train_config.seed)

Random seed set to 42


## Device

In [6]:
device = torch.device("cuda")
print(f"Device: {torch.cuda.get_device_name(device)}")
torch.set_float32_matmul_precision(train_config.matmul_precision)  # Tensor Cores
print(f"MatMul Precision: {train_config.matmul_precision}")

Device: NVIDIA GeForce RTX 4080 SUPER
MatMul Precision: high


# Dataset

In [7]:
dataset_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/shakespeare.txt")
shakespeare_text = load_text(dataset_path)

Loaded text data from /home/pathfinder/projects/PathFinder/datasets/Shakespeare/shakespeare.txt (length: 1115394 characters).


In [8]:
if train_config.debug:
    subset_shakespeare_text = shakespeare_text[:10000]
    print(subset_shakespeare_text)
    shakespeare_text = subset_shakespeare_text

# Tokenizer

In [9]:
char_tokenizer = CharTokenizer()
char_tokenizer.build_vocab(text=shakespeare_text)
vocab_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/vocab.json")
char_tokenizer.save_vocab(vocab_path)
model_config.vocab_size = char_tokenizer.vocab_size

Vocabulary size: 69
Vocabulary saved to /home/pathfinder/projects/PathFinder/datasets/Shakespeare/vocab.json.


In [10]:
if train_config.debug:
    print("Vocabulary:", char_tokenizer.char2idx)

# Preprocessing

In [11]:
train_text, val_text = split_text(shakespeare_text, val_size=0.1)
print(f"Training text length: {len(train_text)} characters")
print(f"Validation text length: {len(val_text)} characters")

Training text length: 1003854 characters
Validation text length: 111540 characters


In [12]:
class TextDataset(Dataset):
    def __init__(self, text: str, tokenizer: CharTokenizer, max_seq_len: int):
        self.encoded = tokenizer.encode(text)
        self.max_seq_len = max_seq_len

    def __len__(self) -> int:
        return len(self.encoded) - self.max_seq_len

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        input_ids = self.encoded[idx:idx + self.max_seq_len]
        target_ids = self.encoded[idx + 1:idx + self.max_seq_len + 1]
        return input_ids, target_ids

def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    target_ids = torch.stack([item[1] for item in batch])
    return {"input_ids": input_ids, "target_ids": target_ids}

train_dataset = TextDataset(train_text, char_tokenizer, model_config.max_seq_len)
val_dataset = TextDataset(val_text, char_tokenizer, model_config.max_seq_len)

train_loader = DataLoader(
    train_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=True,
    num_workers=4
)
val_loader = DataLoader(
    val_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=False,
    num_workers=4
)

In [13]:
if train_config.debug:
    sample_batch = next(iter(train_loader))
    print(f"Sample input IDs: {sample_batch['input_ids'][0]}")
    print(f"Sample target IDs: {sample_batch['target_ids'][0]}")

# Model

In [14]:
# Initialize the model
model = GPT(model_config).to(device)
model = torch.compile(model)
print(model)
print(f"Number of parameters: {model.num_params() / 1e6:.2f}M")

OptimizedModule(
  (_orig_mod): GPT(
    (token_embedding): Embedding(69, 128)
    (positional_encoding): Embedding(128, 128)
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadAttention(
          (Wq): Linear(in_features=128, out_features=128, bias=False)
          (Wkv_down): Linear(in_features=128, out_features=16, bias=False)
          (Wk_up): Linear(in_features=16, out_features=128, bias=False)
          (Wv_up): Linear(in_features=16, out_features=128, bias=False)
          (out_proj): Linear(in_features=128, out_features=128, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): FeedForward(
          (fc1): Linear(in_features=128, out_features=64, bias=False)
          (fc2): Linear(in_features=64, out_features=128, bias=False)
          (ac

# Training

In [15]:
trainer = Trainer(
    model=model,
    train_config=train_config,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    master_process=True
)
trainer.train()

Training: 100%|██████████| 981/981 [01:24<00:00, 11.55it/s, epoch=1, grad_norm=0.2023, loss=1.3613, lr=0.000000]


Grad Norm,██▆▂▂▁▂▂▂▃▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Learning Rate,▂▅██████▇▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁
Train Loss,█▇▇▇▇▇▆▆▅▅▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val Loss,█▅▃▂▂▁▁▁▁▁
Val Perplexity,█▅▂▂▁▁▁▁▁▁
Grad Norm,0.20231
Learning Rate,0
Train Loss,1.36126
Val Loss,1.52853
Val Perplexity,4.61139


## Save the model

In [16]:
if not train_config.debug:
    output_dir = os.path.join(PROJECT_ROOT, "checkpoints", train_config.model_name, train_config.run_name)
    os.makedirs(output_dir, exist_ok=True)
    try:
        model.save_pretrained(
            output_dir,
            safe_serialization=True
        )
        print("Model saved successfully")
    except Exception as e:
        print(f"Error saving model: {e}")
    # Push to Hugging Face Hub
    #model.push_to_hub(
    #    repo_id=f"PathFinderKR/{train_config.model_name}-{train_config.run_name}",
    #    private=True,
    #    use_auth_token=os.environ.get("HUGGINGFACE_TOKEN")
    #)
    #print(f"Model pushed to Hugging Face Hub: PathFinderKR/{train_config.model_name}-{train_config.run_name}")

Model saved successfully


In [17]:
# To load the model later, you can use:
# model = GPT(model_config)
# model = model.from_pretrained(output_dir).to(device)

# Inference

In [18]:
user_prompt = "To be, or not to be, that is the question"
input_ids = char_tokenizer.encode(user_prompt).unsqueeze(0).to(device)
output = model.generate(
    input_ids,
    use_cache=True,
    max_new_tokens=generation_config.max_new_tokens,
    temperature=generation_config.temperature,
    top_k=generation_config.top_k,
    tokenizer=char_tokenizer
)
response = char_tokenizer.decode(output[0].squeeze().cpu().numpy())

s of thy mother,
Angel, if you not, I'll say unto you not so light.

Page:
Ay, farewell:Resetting KV cache

Return you are being bottle, give me.

LUCIO:
What, I know have been, no before me, sweet sound a
peace what will such rememberResetting KV cache
, my sovereign, at is, one shame
And all gives, if you should be gone.

POLIXENES:
She dare noble pure;
And does but you.

BUCKIResetting KV cache
ll you of all
The mark in Rome of Prince, truth, pitch me speak,
We will be to have answer up in good. Thou despairs
When it is Resetting KV cache
prince content?
He have my fellow resign than you ask me not.

RICHARD:
Tell me you die outh.
But all thyself see must brew whatResetting KV cache
he crowns,
And work thy sisters boldness to me griave
The birth free of it, banishment lords.

KING RICHARD III:
With woman--

QResetting KV cache
UEEN ELIZABETH:
My roter's country, do your command, seem of night.
I saw hear thee, fept, and kill'd them all the trebling worlResetting KV cache
f
To be a m

In [19]:
print("=" * 50)
print("User prompt: ")
print(user_prompt)
print("-" * 50)
print("🤖 Model Response:")
print(response)

User prompt: 
To be, or not to be, that is the question
--------------------------------------------------
🤖 Model Response:
To be, or not to be, that is the questions of thy mother,
Angel, if you not, I'll say unto you not so light.

Page:
Ay, farewell:
Return you are being bottle, give me.

LUCIO:
What, I know have been, no before me, sweet sound a
peace what will such remember, my sovereign, at is, one shame
And all gives, if you should be gone.

POLIXENES:
She dare noble pure;
And does but you.

BUCKIll you of all
The mark in Rome of Prince, truth, pitch me speak,
We will be to have answer up in good. Thou despairs
When it is prince content?
He have my fellow resign than you ask me not.

RICHARD:
Tell me you die outh.
But all thyself see must brew whathe crowns,
And work thy sisters boldness to me griave
The birth free of it, banishment lords.

KING RICHARD III:
With woman--

QUEEN ELIZABETH:
My roter's country, do your command, seem of night.
I saw hear thee, fept, and kill'd them

# Speedometer

In [20]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=False,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)

KV cache enabled: False
Latency (Time per token): 2.80 ms
Throughput (Tokens per second): 357.06


In [21]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=True,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)

KV cache enabled: True
Latency (Time per token): 2.66 ms
Throughput (Tokens per second): 376.01
